In [95]:
import json
import pandas as pd
#!pip install openai -U
from openai import OpenAI
from enum import Enum
from typing import Optional, List
from pydantic import BaseModel
from pydantic import Field

In [96]:
with open('../key/key.json') as f:
    k = json.load(f)['key']

In [122]:
# create one time list of 10 human emotions
client = OpenAI(api_key=k)
class Chr_answer(BaseModel):
    Characteristics: List[str] = Field(None, description="Emotions as a list of strings")

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Be a helpful assistant."},
        {"role": "user", "content": "List 10 human emotions."}
    ],
    response_format=Chr_answer,
)

#you can also procees from parsed eg json.loads(completion_song.choices[0].message.parsed.json())
out = json.loads(completion.choices[0].message.content)
out = list(out.values())[0]
emotions = out
print(emotions)

['happiness', 'sadness', 'anger', 'fear', 'disgust', 'surprise', 'trust', 'anticipation', 'joy', 'love']


In [123]:
# class Char(BaseModel): 
#     is_char: bool
#     characteristic: Optional[Characteristic] ?
#fiels is only defined for the first custom class?

#List 10 best selling American clothing brands and, for each brand, list which one of the emotions in the given list are most closely associated with."
client = OpenAI(api_key=k)

class Characteristic(Enum):
    Happiness = 'Happiness'
    Sadness = 'Sadness'
    Anger = 'Anger'
    Fear = 'Fear'
    Surprise = 'Surprise'
    Disgust = 'Disgust'
    Trust = 'Trust'
    Anticipation ='Anticipation'
    Joy ='Joy'
    Love = 'Love'

class BrandAttributes(BaseModel):
    brand_name: str = Field(None, description=" name as str")
    characteristics: List[Characteristic] = Field(None, description=" list of characteristics as str")


class BrandChars(BaseModel):
    a: List[BrandAttributes] = Field(None, description="List of BrandAttributes")
    
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Be a helpful assistant."},
        {"role": "user", "content": "List 10 best selling American clothing brands and, for each brand, list which one of the emotions in the given list are most closely associated with."}
    ],
    response_format=BrandChars,
)

out = json.loads(completion.choices[0].message.content)
print(out)
out = list(out.values())[0]
df = pd.DataFrame(data = [i.values() for i in out], columns = ['Name','characteristic'])
df
#df.set_index('name', inplace=True)
#df.to_dict()

{'a': [{'brand_name': 'Nike', 'characteristics': ['Joy', 'Anticipation']}, {'brand_name': 'Ralph Lauren', 'characteristics': ['Trust', 'Joy']}, {'brand_name': "Levi's", 'characteristics': ['Trust', 'Joy']}, {'brand_name': 'Under Armour', 'characteristics': ['Anticipation', 'Trust']}, {'brand_name': 'Tommy Hilfiger', 'characteristics': ['Trust', 'Joy']}, {'brand_name': 'Gap', 'characteristics': ['Joy', 'Trust']}, {'brand_name': 'Calvin Klein', 'characteristics': ['Love', 'Trust']}, {'brand_name': 'Michael Kors', 'characteristics': ['Joy', 'Love']}, {'brand_name': 'Guess', 'characteristics': ['Surprise', 'Joy']}, {'brand_name': 'Vans', 'characteristics': ['Joy', 'Trust']}]}


,Name,characteristic
0,Nike,"[Joy, Anticipation]"
1,Ralph Lauren,"[Trust, Joy]"
2,Levi's,"[Trust, Joy]"
3,Under Armour,"[Anticipation, Trust]"
4,Tommy Hilfiger,"[Trust, Joy]"
5,Gap,"[Joy, Trust]"
6,Calvin Klein,"[Love, Trust]"
7,Michael Kors,"[Joy, Love]"
8,Guess,"[Surprise, Joy]"
9,Vans,"[Joy, Trust]"


In [124]:
#Take the example song, and list which one of the emotions in the given list are most closely associated with the song.")

client = OpenAI(api_key=k)

class Characteristic(Enum):
    Happiness = 'Happiness'
    Sadness = 'Sadness'
    Anger = 'Anger'
    Fear = 'Fear'
    Surprise = 'Surprise'
    Disgust = 'Disgust'
    Trust = 'Trust'
    Anticipation ='Anticipation'
    Joy ='Joy'
    Love = 'Love'

class SongAttributes(BaseModel):
    song_name: str
    song_lyrics_summary :str = Field(None, description="Summary as str; Don't state the exact lyrics.")
    characteristics: List[Characteristic]

class SongsChars(BaseModel):
    a: List[SongAttributes] = Field(None, description="List of song attributes.")

prompt = """
As prompt, take the song 'Black Balloon', list summary of its lyrics, and a list of the emotions from the given list that are most closely associated with the song.
"""

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Be a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    response_format=SongsChars,
)
out = json.loads(completion.choices[0].message.content)
out = list(out.values())[0]
out_song = out
out_song

[{'song_name': 'Black Balloon',
  'song_lyrics_summary': 'The song reflects on themes of love, loss, and letting go. It discusses how a relationship can be overshadowed by personal struggles and how the external problems influence both individuals.',
  'characteristics': ['Sadness', 'Love', 'Trust']}]

In [ ]:

#OpenAI example
#define 3 categories to be represented to the user as strings in json
#class Category(Enum):
#    violence = "violence"
#    sexual = "sexual"
#    self_harm = "self_harm"

#defining what my data type looks like so it properly imported and exported as json
#class ContentCompliance(BaseModel): 
#    is_violating: bool  #tell me if there is a violation and if so, classify which one of the category classes
#    category: Optional[Category] #the type of the category could be none or category. If non violating, no need to define category
#    explanation_if_violating: Optional[str]
